In [1]:
badscript = '''contract get_fib_n(n:int){
    let last_fib_calculated : int = 0;

    entry get_fib(n: int){
        let result: int = fib(n);
        last_fib_calculated = result;
        let a : int = b;
    }

    func fib(n: int) : int{
        let x : int = last_fib_calculated;
        if (n <= 1) {
            return n;
        }
        else {
            
            let a: int = n - 1;
            let b: int = n - 2;
            return fib(a) + fib(b);
        }
    }

    func fib(n: int) : int{
        let x : int = last_fib_calculated;
    }
}'''

In [2]:
script = '''
contract get_fib_n(n:int){
    let last_fib_calculated: int = 0;

    if (n <= 1) {
            return n;
        }
    
    func fib(n: int) : int{
      
        else {
            let a: int = n - 1;
            let b: int = n - 2;
            return fib(a) + fib(b);
        }
    
    }
    while(n > 0){
        let a : int = 1;
    }
    entry get_fib_n(n: int){
        let result: int = fib(n);
        last_fib_calculated = result;
        return last_fib_calculated;
        
    }
    
}
'''

In [3]:
from lexer.sly_lexer import TzScriptLexer, process_lexer_tokens 
from lexer.lex_token import Token
from parser.tzscript_grammar import TZSCRIPT_GRAMMAR,idx, num, typex, contract, ifx, elsex,equal, plus, minus, star, div,semi, colon, comma, dot, opar, cpar, ocur, ccur,let, func,entry,lessthanequal,greaterthanequal, iniquelaty, lessthan,greaterthan,equalequal, returnx
from parser.slr_parser import SLR1Parser, build_slr_ast

lexer = TzScriptLexer()
lexer_tokens = list(lexer.tokenize(script))
tokens = process_lexer_tokens(lexer_tokens)

parser = SLR1Parser(TZSCRIPT_GRAMMAR, verbose=True)

terminals = [token.token_type for token in tokens]
derivation = parser(terminals, True)

[contract: contract, id: get_fib_n, (: (, id: n, :: :, type: int, ): ), {: {, let: let, id: last_fib_calculated, :: :, type: int, =: =, num: 0, ;: ;, if: if, (: (, id: n, <=: <=, num: 1, ): ), {: {, return: return, id: n, ;: ;, }: }, func: func, id: fib, (: (, id: n, :: :, type: int, ): ), :: :, type: int, {: {, else: else, {: {, let: let, id: a, :: :, type: int, =: =, id: n, -: -, num: 1, ;: ;, let: let, id: b, :: :, type: int, =: =, id: n, -: -, num: 2, ;: ;, return: return, id: fib, (: (, id: a, ): ), +: +, id: fib, (: (, id: b, ): ), ;: ;, }: }, }: }, while: while, (: (, id: n, >: >, num: 0, ): ), {: {, let: let, id: a, :: :, type: int, =: =, num: 1, ;: ;, }: }, entry: entry, id: get_fib_n, (: (, id: n, :: :, type: int, ): ), {: {, let: let, id: result, :: :, type: int, =: =, id: fib, (: (, id: n, ): ), ;: ;, id: last_fib_calculated, =: =, id: result, ;: ;, return: return, id: last_fib_calculated, ;: ;, }: }, }: }, $: EOF]
0 	 S' -> .<program>, 
	 <program> -> .contractid(<param-li

In [4]:
productions, operations = derivation


from visitors.string_rep_visitor import FormatVisitor
ast = build_slr_ast(productions, operations, tokens)
format = FormatVisitor()
print(format.visit(ast))

TypeError: sequence item 3: expected str instance, list found

In [ ]:
from visitors.semantic_check_visitor import SemanticCheckerVisitor
semantic_checker = SemanticCheckerVisitor()
semantic_checker.visit(ast)


fib
fib
n arg id
fib
estoy en last_fib_calculated [<visitors.scope.VariableInfo object at 0x108625b90>, <visitors.scope.VariableInfo object at 0x108626290>, <visitors.scope.VariableInfo object at 0x108627bd0>]


['If statement is not allowed in main level',
 'Return statement is not allowed in main level',
 'Invalid variable n',
 'Else statement is not allowed with outside if statement']